In [69]:
from bs4 import BeautifulSoup
from selenium import webdriver
import random
import html
import requests
import re
from time import sleep
from urllib.parse import quote
import pandas as pd
from concurrent.futures import ProcessPoolExecutor

# Этап 1. Получение ссылок на книги автора

In [45]:
authors = ['Фрай М.', 'Хантер Э.', 'Емец Д.']
source_site = 'https://www.chitai-gorod.ru/search/result.php?q={}&type=author'
sites = {}

In [46]:
def random_sleep(offset=1.5, length=4):
    sleep(random.random() * length + offset)
    
# функция ищет ссылки на книги данного автора
def get_books_for_author(author, driver):
    res = []
    
    driver.get(source_site.format(quote(author, encoding='windows-1251')))
    random_sleep()

    # ищем количество найденных книг
    count_res = int(driver.find_element_by_class_name("count-result__value").text)

    # до тех пор пока на странице не будут все книги мотаем вниз и нажимаем кнопку
    t = driver.find_elements_by_class_name("product-card__link")
    while len(t) != count_res:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        random_sleep()
        try:
            butt = driver.find_element_by_css_selector(
                'body > \
                div.page-wrap > div > \
                div.catalog > div > \
                div.content > div.search__output > \
                div.catalog__btn_show-more > div')
            butt.click()
            random_sleep()
        except:
            pass
        t = driver.find_elements_by_class_name("product-card__link")

    # записываем в результат найденные книги
    for el in t:
        res.append(el.get_attribute('href'))

    return res

In [47]:
with webdriver.Chrome() as driver:
    for author in authors:
        sites[author] = get_books_for_author(author, driver)

# Этап 2. Получение информации о книгах

In [98]:
def extract_book_info(site):
    soup = BeautifulSoup(requests.get(site).text, 'html.parser')
    nums = re.compile(r"[+-]?\d+(?:\.\d+)?")
    
    book_info = {
        "ID карточки": site.split('/')[-2],
        "Название": soup.find('h1', class_='product__title', itemprop='name').text,
        "Автор": soup.find('a', class_='link product__author').text.replace(
            '\t', '').replace('\n', '').strip(),
        "Рейтинг": nums.search(soup.find('div', class_='rating').text).group(0),
        "Голоса": nums.findall(soup.find('div', class_='rating').text)[1],
        "Цена": soup.find('div', class_='price').text.replace(
                    '\t', '').replace('\n', '').strip().split()[0] \
                if soup.find('div', class_='price') is not None
                else None,
    }
    
    table = soup.find('div', class_='product__props')
    rows = table.find_all('div', class_='product-prop')
    
    for row in rows:
        l = row.find('div', class_='product-prop__title').text.replace('\t', '').replace('\n', '')
        v = row.find('div', class_='product-prop__value').text.replace('\t', '').replace('\n', '')
        book_info[l.strip()] = v.strip()
    
    book_info["Обложка"] = soup.find('img', itemprop='image').attrs['src']
    
    return book_info

In [99]:
sites_l = list(sites.values())
sites_v = sites_l[0] + sites_l[1] + sites_l[2]

In [100]:
%%time
result = []

with ProcessPoolExecutor(max_workers=5) as executor:
    result = list(executor.map(extract_book_info, sites_v))

df = pd.DataFrame(result)
df.sort_values(by='ID карточки', inplace=True)

CPU times: user 485 ms, sys: 148 ms, total: 633 ms
Wall time: 1min 27s


In [103]:
df.head(4).T

,432,142,230,433
ID карточки,1002042,1004167,1004240,1006687
ID товара:,2607833,2608544,2608971,2611099
ISBN,9785699973705,9785171053802,9785001111900,9785699973712
Автор,Емец Д.,Фрай М.,Хантер Э.,Емец Д.
Возраст,12+,18+,6+,12+
Год издания,2017,2017,2017,2017
Голоса,0,4,1,0
Издательство,Эксмо,АСТ,Абрис,Эксмо
Кол-во страниц,416,288,350,416
Название,Лед и пламя Тартара,Новая чайная книга,Расколотый прайд,Первый эйдос


In [102]:
with open('hw_4.csv', mode='w', encoding='utf-8') as f_csv:
    df.to_csv(f_csv, index=False)